In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
import os
import json
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics, is_outlier
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Removing outliers

In [6]:
print(numeric_columns)
mean_per_column = {col: data[col].mean() for col in numeric_columns}
std_per_column = {col: data[col].std() for col in numeric_columns}


['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']


In [7]:
print(f"Before removing outliers: {data.shape}")
if REMOVE_OUTLIERS:
    data = data[~data.apply(is_outlier, axis=1, numeric_columns=numeric_columns, mean_per_column=mean_per_column, std_per_column=std_per_column)]
print(f"After removing outliers: {data.shape}")

Before removing outliers: (10000, 31)
After removing outliers: (9291, 31)


Converting non-numeric features to numerical features

In [8]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9994  14.026668   0.38    0        5   61.4   57.0    653  4.64  4.67  2.86   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [9]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9994  14.026668   0.38    0        5   61.4   57.0    653  4.64  4.67  2.86   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

Data splitting:
- Split the entire dataset into training and testing sets first.
- Use the training set to generate folds (one validation and the rest training folds) (K-Fold Cross Validation)

In [10]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=REPRODUCIBILITY_SEED)
print(f"Training set size: {len(train_data)} | Test set size: {len(test_data)}")    
print()


Training set size: 7432 | Test set size: 1859



In [11]:
normaliser = Normaliser()
os.makedirs(TRAINING_STATISTICS_DIR, exist_ok=True)
stats_for_each_column = {}
for column in numeric_columns:
    print(data[column])
    train_data_column_mean = normaliser.calculate_mean(train_data[column])
    train_data_column_std = normaliser.calculate_std(train_data[column])

    train_data[column] = normaliser.standardise(train_data[column], mean=train_data_column_mean, std=train_data_column_std)

    stats_for_each_column[column] = {
        "mean": train_data_column_mean,
        "std": train_data_column_std
    }

    # Normalise test data using the mean and std of the training data
    test_data[column] = normaliser.standardise(test_data[column], mean=train_data_column_mean, std=train_data_column_std)
    print("after", train_data[column])

with open(f"{TRAINING_STATISTICS_DIR}/stats.json", "w") as f:
    json.dump(stats_for_each_column, f)

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9994    0.38
9995    0.33
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 9291, dtype: float64
after 1446   -1.091530
4333    0.630628
802    -1.091530
1344   -0.756666
5113   -0.613152
          ...   
6170    1.778733
5580   -0.063019
5802    0.606709
926     0.798060
7813   -1.043692
Name: carat, Length: 7432, dtype: float64
0       62.3
1       60.5
2       60.7
3       61.2
4       62.4
        ... 
9994    61.4
9995    62.6
9997    57.9
9998    62.0
9999    59.9
Name: depth, Length: 9291, dtype: float64
after 1446   -0.036532
4333    0.587798
802     0.587798
1344    0.275633
5113    0.275633
          ...   
6170    0.587798
5580   -1.207151
5802    1.290170
926    -0.036532
7813    0.197592
Name: depth, Length: 7432, dtype: float64
0       56.0
1       59.0
2       58.0
3       58.0
4       54.0
        ... 
9994    57.0
9995    57.0
9997    61.0
9998    54.0
9999    59.0
Name: table, Leng

In [12]:
kfold_data = get_kfold_data(data=train_data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (5945, 37) | 79.99%
Validation shape: (1487, 37) | 20.01%
Index(['outcome', 'carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5',
       'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10',
       'colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J'],
      dtype='object')
Index(['outcome', 'carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5',
       'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10',
       'colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J'],
      dtype='object')

Fold: 1/5
Train shape: (5945, 37) | 79.99%
Validation shape: (1487, 37) | 20.01%
Index(['outcome', 'carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', '

Define models and hyperparameter tuning objectives for each model

In [13]:
models = {
        "linear_regression": LinearRegression,
        "lasso": Lasso,
        "ridge": Ridge,
        "xgb": xgb.XGBRegressor,
        "random_forest": RandomForestRegressor,
        "gradient_boosting": GradientBoostingRegressor,
        "ada_boost": AdaBoostRegressor,
        "lgbm": lgb.LGBMRegressor
        }

# Must contain the one-hot encoded columns (otherwise this does not make sense)
minimum_features = [feature for feature in data.columns if feature.startswith("colour")] 

def objective(model_type, trial, x_train, y_train, x_val, y_val):
    if model_type == LinearRegression:
        parameters = {
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        }
    elif model_type == Lasso:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "selection": trial.suggest_categorical("selection", ["cyclic", "random"]),
            "warm_start": trial.suggest_categorical("warm_start", [True, False]),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == Ridge:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "solver": trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "positive": False,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == xgb.XGBRegressor:
        parameters = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "n_estimators": 100,
            "eta": trial.suggest_float("eta", 1e-2, 0.2, log=True),
            "gamma": trial.suggest_float("gamma", 1e-8, 10, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == RandomForestRegressor:
        parameters = {
            "n_estimators": 100,
            "criterion": trial.suggest_categorical("criterion", ["absolute_error", "squared_error"]),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": True,
            "oob_score": False,
            "n_jobs": -1,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == GradientBoostingRegressor:
        parameters = {
            "n_estimators": 100,
            "loss": trial.suggest_categorical("loss", ["absolute_error", "squared_error", "huber", "quantile"]),
            "criterion": trial.suggest_categorical("criterion", ["friedman_mse", "squared_error"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 2**10),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == AdaBoostRegressor:
        parameters = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == lgb.LGBMRegressor:
        parameters = {
                    "objective": "regression",
                    "metric": "rmse",
                    "n_estimators": 100,
                    "verbosity": -1,
                    "bagging_freq": 1,
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                    "seed": REPRODUCIBILITY_SEED
        }
    
    # Wrapper method for feature selection
    if USE_FEATURE_SELECTION:
        selected_features = minimum_features.copy()
        train_columns = x_train.columns.tolist() 
        for feature in train_columns:
            if feature.startswith("colour"):
                continue
            if trial.suggest_categorical(feature, [0, 1]) == 1:
                selected_features.append(feature)
        if len(selected_features) == 0:
            return float("inf")
        
        x_train = x_train[selected_features]
        x_val = x_val[selected_features]

    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse

In [14]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

if os.path.exists("model_best_hyperparameters"):
    raise Exception("Directory for best hyperparameters already exists. Please delete it before running this script.")

os.makedirs("model_best_hyperparameters")

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    
    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")
    training_features = set(train_x.columns.tolist())


    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        print("G", train_x.columns)
        print("G", val_x.columns)
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=N_TRIALS)
        
        # Train model with best hyperparameters
        best_trial = study.best_trial 

        if USE_FEATURE_SELECTION:
            best_fold_params = {} # Hyperparameters
            best_selected_features = minimum_features.copy() # Features selected by the model
            for param in best_trial.params:
                if param in training_features: # i.e., if the parameter is a feature
                    if best_trial.params[param] == 1:
                        best_selected_features.append(param)
                else: # A hyperparameter
                    best_fold_params[param] = best_trial.params[param]
        else:
            best_fold_params = best_trial.params
            best_selected_features = train_x.columns.tolist() # All feature columns
        print("P", best_fold_params)
        print("F", best_selected_features)
        
        # Select the best features
        print(train_x.columns)
        print(val_x.columns)
        train_x = train_x[best_selected_features]
        val_x = val_x[best_selected_features]

        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)
        
        # Save the best hyperparameters for this model at this fold.
        os.makedirs(f"model_best_hyperparameters/{model_name}", exist_ok=True)
        with open(f"model_best_hyperparameters/{model_name}/fold_{fold+1}.json", "w") as f:
            json.dump(best_fold_params, f)
        with open(f"model_best_hyperparameters/{model_name}/fold_{fold+1}_selected_features.json", "w") as f:
            json.dump(best_selected_features, f)

        # Calculate metrics
        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()


[I 2025-02-18 23:31:54,247] A new study created in memory with name: no-name-daf46459-f1c1-4665-8554-59cecb29c913
[I 2025-02-18 23:31:54,257] Trial 0 finished with value: 10.724077215002952 and parameters: {'fit_intercept': False, 'carat': 0, 'cut': 0, 'clarity': 0, 'depth': 1, 'table': 0, 'price': 0, 'x': 1, 'y': 1, 'z': 1, 'a1': 1, 'a2': 1, 'a3': 0, 'a4': 0, 'a5': 1, 'b1': 1, 'b2': 0, 'b3': 1, 'b4': 1, 'b5': 1, 'a6': 1, 'a7': 1, 'a8': 1, 'a9': 0, 'a10': 1, 'b6': 0, 'b7': 0, 'b8': 1, 'b9': 1, 'b10': 0}. Best is trial 0 with value: 10.724077215002952.
[I 2025-02-18 23:31:54,267] Trial 1 finished with value: 12.146226373218816 and parameters: {'fit_intercept': True, 'carat': 0, 'cut': 1, 'clarity': 1, 'depth': 0, 'table': 1, 'price': 0, 'x': 0, 'y': 0, 'z': 0, 'a1': 0, 'a2': 1, 'a3': 0, 'a4': 0, 'a5': 1, 'b1': 1, 'b2': 0, 'b3': 1, 'b4': 0, 'b5': 0, 'a6': 1, 'a7': 1, 'a8': 1, 'a9': 0, 'a10': 1, 'b6': 0, 'b7': 1, 'b8': 1, 'b9': 1, 'b10': 1}. Best is trial 0 with value: 10.724077215002952.

G Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')
G Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')


[I 2025-02-18 23:31:54,341] Trial 11 finished with value: 10.792532716501293 and parameters: {'fit_intercept': False, 'carat': 0, 'cut': 0, 'clarity': 0, 'depth': 1, 'table': 0, 'price': 0, 'x': 1, 'y': 1, 'z': 1, 'a1': 1, 'a2': 1, 'a3': 0, 'a4': 1, 'a5': 0, 'b1': 0, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'a6': 1, 'a7': 1, 'a8': 1, 'a9': 0, 'a10': 1, 'b6': 0, 'b7': 1, 'b8': 1, 'b9': 1, 'b10': 0}. Best is trial 0 with value: 10.724077215002952.
[I 2025-02-18 23:31:54,353] Trial 12 finished with value: 10.792532716501293 and parameters: {'fit_intercept': False, 'carat': 0, 'cut': 0, 'clarity': 0, 'depth': 1, 'table': 0, 'price': 0, 'x': 1, 'y': 1, 'z': 1, 'a1': 1, 'a2': 1, 'a3': 0, 'a4': 1, 'a5': 0, 'b1': 0, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'a6': 1, 'a7': 1, 'a8': 1, 'a9': 0, 'a10': 1, 'b6': 0, 'b7': 1, 'b8': 1, 'b9': 1, 'b10': 0}. Best is trial 0 with value: 10.724077215002952.
[I 2025-02-18 23:31:54,366] Trial 13 finished with value: 10.792532716501293 and parameters: {'fit_intercept':

P {'fit_intercept': True}
F ['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J', 'carat', 'cut', 'depth', 'table', 'price', 'a1', 'a3', 'a4', 'b1', 'b3', 'a6', 'a8', 'a9', 'b6', 'b7', 'b10']
Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')
Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')
Fold: 1/5
Model name: linear_regression
MAE: 8.445879572969561
MSE: 111.9218226393591
RMSE: 10.579311066386087
PCC: 0.5589595

[I 2025-02-18 23:31:55,734] Trial 20 finished with value: 11.097292532693897 and parameters: {'alpha': 0.09926204309020928, 'fit_intercept': False, 'selection': 'cyclic', 'warm_start': False, 'carat': 1, 'cut': 1, 'depth': 1, 'table': 1, 'price': 0, 'a1': 1, 'a3': 0, 'a4': 0, 'b1': 1, 'b3': 0, 'a6': 1, 'a8': 1, 'a9': 1, 'b6': 0, 'b7': 0, 'b10': 1}. Best is trial 18 with value: 11.024663517842555.
[I 2025-02-18 23:31:55,743] Trial 21 finished with value: 11.090664797544816 and parameters: {'alpha': 0.09390809003723272, 'fit_intercept': False, 'selection': 'cyclic', 'warm_start': False, 'carat': 1, 'cut': 1, 'depth': 1, 'table': 1, 'price': 0, 'a1': 1, 'a3': 0, 'a4': 0, 'b1': 1, 'b3': 0, 'a6': 1, 'a8': 1, 'a9': 1, 'b6': 0, 'b7': 0, 'b10': 1}. Best is trial 18 with value: 11.024663517842555.
[I 2025-02-18 23:31:55,753] Trial 22 finished with value: 11.054490520609177 and parameters: {'alpha': 0.06070009329063373, 'fit_intercept': False, 'selection': 'cyclic', 'warm_start': False, 'carat':

...

P {'n_estimators': 81, 'loss': 'exponential', 'learning_rate': 0.059114460818983805}
F ['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J', 'depth', 'a1', 'b3']
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b3', 'a7'],
      dtype='object')
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b3', 'a7'],
      dtype='object')


[I 2025-02-18 23:41:19,905] A new study created in memory with name: no-name-8974f66c-6854-4560-90bf-a055388bded8


Fold: 5/5
Model name: ada_boost
MAE: 7.917338153236487
MSE: 97.87314530000232
RMSE: 9.89308573196464
PCC: 0.6418833059647899
Spearman R: 0.6458355858375695
R2 Score: 0.4109287753330947

G Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b3'],
      dtype='object')
G Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b3'],
      dtype='object')


[I 2025-02-18 23:41:20,376] Trial 0 finished with value: 10.70386303441481 and parameters: {'learning_rate': 0.019762149818082712, 'num_leaves': 750, 'subsample': 0.7017567840600334, 'colsample_bytree': 0.7600219417258738, 'min_data_in_leaf': 6, 'depth': 1, 'a1': 1, 'b3': 0}. Best is trial 0 with value: 10.70386303441481.
[I 2025-02-18 23:41:20,421] Trial 1 finished with value: 12.794183653537388 and parameters: {'learning_rate': 0.09293087206104114, 'num_leaves': 252, 'subsample': 0.7909359212724346, 'colsample_bytree': 0.7352428223894203, 'min_data_in_leaf': 86, 'depth': 0, 'a1': 1, 'b3': 0}. Best is trial 0 with value: 10.70386303441481.
[I 2025-02-18 23:41:20,541] Trial 2 finished with value: 10.416802797968653 and parameters: {'learning_rate': 0.06653903026289387, 'num_leaves': 869, 'subsample': 0.6690256396122187, 'colsample_bytree': 0.5931102397388968, 'min_data_in_leaf': 26, 'depth': 1, 'a1': 0, 'b3': 1}. Best is trial 2 with value: 10.416802797968653.
[I 2025-02-18 23:41:20,61

P {'learning_rate': 0.04880270694341699, 'num_leaves': 5, 'subsample': 0.4313420532575576, 'colsample_bytree': 0.7958031635051172, 'min_data_in_leaf': 31}
F ['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J', 'depth', 'a1', 'b3']
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b3'],
      dtype='object')
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b3'],
      dtype='object')
Fold: 5/5
Model name: lgbm
MAE: 7.861164966457033
MSE: 96.68296334655786
RMSE: 9.83274953136496
PCC: 0.6485259748717046
Spearman R: 0.651131264068086
R2 Score: 0.41809214929786254



Compute average scores and rank models by R2 score

In [15]:
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])

model_scores = dict(sorted(model_scores.items(), key=lambda x: x[1]["r2_score"], reverse=True))

In [16]:
# Print results
for i, (model_name, model_metrics) in enumerate(model_scores.items()):
    print(f"No.{i+1} Model: {model_name}")
    for metric, score in model_metrics.items():
        print(f"{metric}: {score}")
    print()

No.1 Model: xgb
mae: 7.450683901007574
mse: 87.88724788979896
rmse: 9.373804283174959
pcc: 0.6751894815076779
spearman_r: 0.6801768309463323
r2_score: 0.45423330161859504

No.2 Model: random_forest
mae: 7.5175070178551575
mse: 89.39355785728699
rmse: 9.454062976153274
pcc: 0.6702822643389339
spearman_r: 0.6738402406125337
r2_score: 0.4448688635979587

No.3 Model: gradient_boosting
mae: 7.630175740803702
mse: 91.79799517979038
rmse: 9.579690082987636
pcc: 0.657636262110951
spearman_r: 0.6610262695291291
r2_score: 0.43002805114779286

No.4 Model: ada_boost
mae: 7.70381967720981
mse: 93.72728778239878
rmse: 9.68028005451287
pcc: 0.647475657547147
spearman_r: 0.6534548100068588
r2_score: 0.4180225995839904

No.5 Model: lgbm
mae: 7.72913356164467
mse: 94.76162519198378
rmse: 9.733948386527386
pcc: 0.6465235259178278
spearman_r: 0.6490956734093304
r2_score: 0.41129347840788133

No.6 Model: lasso
mae: 8.454680495643604
mse: 112.43587459054038
rmse: 10.602313984635568
pcc: 0.5502258276234644
s